In [10]:
# Importing libraries
import gym
import numpy as np
from itertools import count
from collections import namedtuple
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import pandas as pd

In [2]:
env = gym.make('CartPole-v0')
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
# You can move either left or right to balance the pole
# Lets implement the Actor critic network
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(4, 128) # 4 because there are 4 parameters as the observation space
        self.actor = nn.Linear(128, 2) # 2 for the number of actions
        self.critic = nn.Linear(128, 1) # Critic is always 1
        self.saved_actions = []
        self.rewards = []
    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_prob = F.softmax(self.actor(x), dim=-1)
        state_values = self.critic(x)
        return action_prob, state_values

def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()

def finish_episode():
    # We calculate the losses and perform backprop in this function
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses =[]
    returns = []
    
    for r in model.rewards[::-1]:
        R = r + 0.99 * R # 0.99 is our gamma number
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    
    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()
        
        policy_losses.append(-log_prob * advantage)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))
    
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    
    loss.backward()
    optimizer.step()
    
    del model.rewards[:]
    del model.saved_actions[:]

In [3]:
model = ActorCritic()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [4]:
def train():
    running_reward = 10
    rew = []
    for i_episode in count(): # We need around this much episodes
        state = env.reset()
        ep_reward = 0
        for t in range(1, 10000):
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break
        running_reward = 0.05 * ep_reward + (1-0.05) * running_reward
        finish_episode()
        if i_episode % 10 == 0: # We will print some things out
            print("Episode {}\tLast Reward: {:.2f}\tAverage reward: {:.2f}".format(i_episode, ep_reward, running_reward))
        rew.append(ep_reward)
        if running_reward > env.spec.reward_threshold:
            print("Solved, running reward is now {} and the last episode runs to {} time steps".format(
                    running_reward, t))
            break
    return(rew)

In [5]:
rew = train()

Episode 0	Last Reward: 19.00	Average reward: 10.45
Episode 10	Last Reward: 9.00	Average reward: 12.77
Episode 20	Last Reward: 104.00	Average reward: 22.70
Episode 30	Last Reward: 80.00	Average reward: 33.52
Episode 40	Last Reward: 13.00	Average reward: 57.37
Episode 50	Last Reward: 16.00	Average reward: 40.59
Episode 60	Last Reward: 13.00	Average reward: 29.71
Episode 70	Last Reward: 11.00	Average reward: 23.98
Episode 80	Last Reward: 27.00	Average reward: 22.19
Episode 90	Last Reward: 27.00	Average reward: 22.55
Episode 100	Last Reward: 38.00	Average reward: 25.60
Episode 110	Last Reward: 47.00	Average reward: 35.05
Episode 120	Last Reward: 200.00	Average reward: 87.74
Episode 130	Last Reward: 200.00	Average reward: 132.79
Episode 140	Last Reward: 101.00	Average reward: 136.01
Episode 150	Last Reward: 189.00	Average reward: 112.01
Episode 160	Last Reward: 200.00	Average reward: 121.19
Episode 170	Last Reward: 200.00	Average reward: 152.81
Episode 180	Last Reward: 200.00	Average reward

In [19]:
np.savetxt("a2c_cp.csv",rew,delimiter =", ",fmt ='% d')